In [1]:
import pandas as pd
from sqlalchemy import *

In [2]:
def load_csv_to_db(engine, csv_path, table_name):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, engine, index=False, if_exists="replace")

In [3]:
def compare_tables(engine, left_name, right_name, key_col):
    meta = MetaData()
    meta.reflect(bind=engine, only=[left_name, right_name])
    left  = Table(left_name,  meta, autoload_with=engine)
    right = Table(right_name, meta, autoload_with=engine)
    all_cols     = list(left.columns.keys())
    non_key_cols = [c for c in all_cols if c != key_col]
    removed_cols = [left.c[c] for c in all_cols] + [
        literal("REMOVED").label("change_type")
    ]
    removed_stmt = (
        select(*removed_cols)
        .select_from(outerjoin(left, right, left.c[key_col] == right.c[key_col]))
        .where(right.c[key_col] == None)
    )
    added_cols = [right.c[c] for c in all_cols] + [
        literal("ADDED").label("change_type")
    ]
    added_stmt = (
        select(*added_cols)
        .select_from(outerjoin(right, left, left.c[key_col] == right.c[key_col]))
        .where(left.c[key_col] == None)
    )
    full_rows_df = pd.read_sql(
        removed_stmt.union_all(added_stmt),
        con=engine
    )
    col_dfs = []
    for col in non_key_cols:
        stmt = (
    select(left.c[key_col].label(key_col),literal(col).label("column_changed"),left.c[col].label("old_value"),right.c[col].label("new_value"),)
            .select_from(left.join(right, left.c[key_col] == right.c[key_col]))
            .where(or_(left.c[col] != right.c[col],and_( left.c[col].is_(None),right.c[col].isnot(None)),and_( left.c[col].isnot(None),right.c[col].is_(None)),
                )
            )
        )
        df_col = pd.read_sql(stmt, con=engine)
        if not df_col.empty:
            col_dfs.append(df_col)
            
    if col_dfs:
        col_changes_df = pd.concat(col_dfs, ignore_index=True)
    else:
        col_changes_df = pd.DataFrame(
            columns=[key_col, "column_changed", "old_value", "new_value"]
        )

    return full_rows_df, col_changes_df
            

In [4]:
def compare_snapshots(
    left_csv,
    right_csv,
    key_col="product_id",
    engine_url="sqlite:///:memory:",
    left_name="day1",
    right_name="day2"
):
    engine = create_engine(engine_url)
    load_csv_to_db(engine, left_csv,  left_name)
    load_csv_to_db(engine, right_csv, right_name)
    return compare_tables(engine, left_name, right_name, key_col)

In [5]:
full_diffs, col_diffs = compare_snapshots(
    left_csv   = "products_day1.csv",
    right_csv  =  "products_day2.csv",
    key_col    = "product_id",
    engine_url = "sqlite:///:memory:",
    left_name  = "day1",
    right_name = "day2"
)

In [6]:
full_diffs = full_diffs.sort_values(by="product_id")
full_diffs.to_csv('Changes in Full Rows.csv', index=False)
print("Changes in Full Rows (Added or Removed):")
display(full_diffs)
col_diffs = col_diffs.sort_values(by="product_id")
col_diffs.to_csv('Changes in Column Values.csv', index=False)
print("Column-Level Differences for Matching product_ids:")
display(col_diffs)

Changes in Full Rows (Added or Removed):


,product_id,name,category,price,stock,change_type
0,106,Notebook,Stationery,2.99,300,REMOVED
1,107,Smart Watch,Wearables,99.00,50,ADDED


Column-Level Differences for Matching product_ids:


,product_id,column_changed,old_value,new_value
0,101,price,25.99,23.99
2,102,stock,200.00,180.00
1,105,price,35.00,37.00
